<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_%EB%B3%80%ED%98%95_%EB%93%80%EC%96%BC%EB%AA%A8%EB%A9%98%ED%85%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [53]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

In [54]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = '변형 듀얼모멘텀 '

# 전략에 사용할 ETF 명 리스트에 담기
etf_aggressive = ['SPY', 'EFA']
etf_defensive = ['SHY', 'IEF', 'TLT', 'TIP', 'LQD', 'HYG', 'BWX', 'EMB']
etf_bil = ['BIL']

# 자산군 리스트 더하기, 중복 제거
etf_order = etf_aggressive + etf_defensive + etf_bil

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  11 of 11 completed


,SPY,EFA,SHY,IEF,TLT,TIP,LQD,HYG,BWX,EMB,BIL
2023-08-25,439.9700,70.2300,80.9300,93.6900,95.2200,105.4300,104.9600,74.6100,21.9700,84.8800,91.7700
2023-08-24,436.8900,69.7600,80.9800,93.7200,94.9100,105.4500,104.9000,74.3000,22.0000,84.7500,91.7500
2023-08-23,443.0300,70.7300,81.0300,94.0700,95.5400,105.9300,105.2600,74.7100,22.1500,85.3300,91.7200
2023-08-22,438.1500,69.9700,80.8800,93.0600,93.2300,104.9800,103.8000,74.1100,21.8700,83.9800,91.7000
2023-08-21,439.3400,70.1000,80.9200,93.0200,92.5200,104.8000,103.4800,74.0700,21.8700,83.7500,91.6900
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-23,405.7246,62.2253,80.4644,99.1994,108.7349,111.6503,106.7934,71.9855,22.2425,82.8663,88.0619
2022-08-22,406.7085,62.1960,80.4254,99.4430,109.3367,111.4665,106.6779,71.8062,22.2918,82.0002,88.0812
2022-08-19,415.3574,63.2718,80.5327,99.8816,109.7055,111.6793,107.3997,72.6273,22.4788,82.6093,88.0715
2022-08-18,421.0150,64.1521,80.5913,100.7491,111.5009,111.8728,108.6893,73.3729,22.7250,83.8846,88.0715


In [55]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} 전통 듀얼모멘텀.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')


# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-08-27 변형 듀얼모멘텀 ### ETF 현재-1-3-6-12 개월 가격

,SPY,EFA,SHY,IEF,TLT,TIP,LQD,HYG,BWX,EMB,BIL
today/2023-08-25,439.9700,70.2300,80.9300,93.6900,95.2200,105.4300,104.9600,74.6100,21.9700,84.8800,91.7700
1mo_ago/2023-07-27,452.4900,73.9100,80.7783,95.0691,99.0271,106.4999,106.4844,74.5349,22.7270,86.1666,91.3861
3mo_ago/2023-05-25,413.1154,70.1540,80.8452,96.0459,99.4699,106.5663,104.9757,72.4812,22.1246,82.8863,90.5876
6mo_ago/2023-02-24,393.4116,67.7972,79.8069,94.0201,99.4046,104.8704,103.7630,71.9895,21.8837,83.0765,89.5944
12mo_ago/2022-08-24,407.0234,62.3035,80.3766,98.8680,107.9391,111.6309,106.4758,72.0704,22.1835,82.7902,88.0908


In [56]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)

### 2023-08-27 변형 듀얼모멘텀 ### ETF 1-3-6-12 개월 수익률

,SPY,EFA,SHY,IEF,TLT,TIP,LQD,HYG,BWX,EMB,BIL
1mo_ago/2023-07-27,-2.7669,-4.9790,0.1878,-1.4506,-3.8445,-1.0046,-1.4316,0.1008,-3.3307,-1.4932,0.4201
3mo_ago/2023-05-25,6.5005,0.1083,0.1049,-2.4529,-4.2726,-1.0663,-0.0150,2.9371,-0.6988,2.4053,1.3052
6mo_ago/2023-02-24,11.8345,3.5883,1.4072,-0.3510,-4.2097,0.5336,1.1536,3.6401,0.3945,2.1708,2.4282
12mo_ago/2022-08-24,8.0945,12.7223,0.6885,-5.2373,-11.7836,-5.5549,-1.4236,3.5237,-0.9622,2.5242,4.1766


In [67]:
# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 채권에 투자
str_is_bond = 'True'
if ror_12mo['SPY'].iloc[0] < ror_12mo['BIL'].iloc[0] and ror_12mo['EFA'].iloc[0] < ror_12mo['BIL'].iloc[0]:
  str_is_bond = 'False'

# 채권 중 수익률 상위 3개에 각 33.3% 씩 투자! 그 중 6개월 수익률이 -이면 현금 보유
str_bond_top_3 = str_today + str_strategy + '### 6개월 수익률 안전자산 상위 3개'
ma_6mo_defensive_top_3 = ror_6mo[etf_defensive].transpose().nlargest(3, str(df_etf.index[0].date())).transpose()
display(Markdown(str_bond_top_3))
display(ma_6mo_defensive_top_3)

# 안전자산 중 0 이상인 것만 필터링하기
defensive_etf_list_text = ''

if str_is_bond == 'True':
  filtered_index = ma_6mo_defensive_top_3.columns[(ma_6mo_defensive_top_3 < 0).any()]
  df_etf_defensive_positive_index = ma_6mo_defensive_top_3.drop(filtered_index, axis=1).columns
  defensive_etf_list_text = ' '.join([str(asset) for asset in df_etf_defensive_positive_index])


## 결과 출력
display(Markdown(str_today[1:] + str_strategy))

str_etf = "## 12개월 수익률 기준, 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? " + str_is_bond
display(Markdown(str_etf))

# 아니라면 SPY, EFA 중 12개월 수익률이 더 높은 것 표시
str_higher_stock = ''

if str_is_bond == 'False':
  if ror_12mo['SPY'].iloc[0] > ror_12mo['EFA'].iloc[0]:
    str_higher_stock = 'SPY'
  else:
    str_higher_stock = 'EFA'

str_higher_stock_text = "## 투자할 공격자산: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# 안전자산 중 6개월 수익률 0 이상 출력
str_etf_defensive_positive = '## 투자할 안전자산: ' + defensive_etf_list_text
display(Markdown(str_etf_defensive_positive))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_etf}\n')
  f.write(f'{str_higher_stock_text}\n')

### 2023-08-27 변형 듀얼모멘텀 ### 6개월 수익률 안전자산 상위 3개

,HYG,EMB,SHY
2023-08-25,3.6401,2.1708,1.4072


## 2023-08-27 변형 듀얼모멘텀 

## 12개월 수익률 기준, 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? False

## 투자할 공격자산: EFA

## 투자할 안전자산: 